In [2]:
import pandas as pd
import requests, re
from requests.adapters import HTTPAdapter, Retry
from urllib.parse import urlparse, parse_qs, urlencode

In [4]:
df = pd.read_csv('input/enzymes_description.csv')
df

,id_seq,enzyme,sequence,uniprot_id,pdb_id,ec_number,genbankID,genes_names,is_canon,T_opt,ph_opt
0,0,PHB-DEPOLYMERASE,MAFNFIRAAAAGAAIALCGVASVHAAVNLPALKIDKTQTTVSGLSS...,Q9ZNH8,NaN,NaN,AB015309.1,NaN,True,NaN,NaN
1,1,PHB-DEPOLYMERASE,MAFNFIRAAAAGAAMALCGVGSVHAAVNLPALKIDKTQTTVSGLSS...,O05527,NaN,NaN,AB003186.1,phaZCac,True,NaN,NaN
2,2,CARBOXYLIC ESTER HYDROLASE,MNSRSLTKAIRFPTILALAGFSVLGACGGSDNDSSSNNQGAPAVAI...,Q9WX47,NaN,NaN,AB009606.1,pudA,True,NaN,NaN
3,3,PLA-DEPOLYMERASE,MRKLKLLLMVCMSMVFIFTLPGMGQSLKASAATERTPIVFVHGLTG...,Q83VD0,NaN,NaN,AB093482.1,PlaA,True,NaN,NaN
4,4,"1,2-DIHYDROXY-3,5-CYCLOHEXADIENE-1,4-DICARBOXY...",MTIVHRRLALAIGDPHGIGPEIALKALQQLSATERSLIKVYGPWSA...,Q5D0X4,NaN,NaN,AY923836.1,tphB,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
942,812,CHITINASE-L,MKLFIALVGLLALAKALPAVTHSKVLCYYDSRSYVRESQARMLPLD...,Q9GV28,NaN,NaN,NaN,EN03,False,NaN,NaN
943,813,HEMAGGLUTININ-ESTERASE-FUS,EKIKICLQKQVNSSFSLHNGFGGNLYATEEKRMFELVKPKAGASVL...,P17004,NaN,NaN,NaN,HE,False,NaN,NaN
944,814,HEMAGGLUTININ-ESTERASE-FUS,EKIKICLQKQVNSSFSLHNGFGGNLYATEEKRMFELVKPKAGASVL...,P17005,NaN,NaN,NaN,HE,False,NaN,NaN
945,815,HEMAGGLUTININ-ESTERASE-FUS,EKIKICLQKQVNSSFSLHNGFGGNLYATEEKRMFELVKPKAGASVL...,P17003,NaN,NaN,NaN,HE,False,NaN,NaN


In [5]:
df = df[df["enzyme"] == "unknown_enzyme"]
df

,id_seq,enzyme,sequence,uniprot_id,pdb_id,ec_number,genbankID,genes_names,is_canon,T_opt,ph_opt
95,62,unknown_enzyme,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN
96,78,unknown_enzyme,MMNVLTKCKLALGIIAIFFSLPSFAVPCSDCSNGFERGQVPRVDQL...,A0A1Z2SIQ1_VIBGA,7Z6B,3.1.1.1,NaN,NaN,True,NaN,NaN
97,79,unknown_enzyme,MELGTLEGSEFAGGGGDGGDGGCTSDCGFQRGPDPTVSFLEASSGP...,OK558825,7VMD,3.1.1.1,NaN,NaN,True,NaN,NaN
98,80,unknown_enzyme,MGGGGSGGGNNGGGGGCEADCGYERGPDPSVSLLEASTGPFSVRTS...,OK558824,7VPA,3.1.1.1,NaN,NaN,True,NaN,NaN
99,81,unknown_enzyme,MINRTLPNSLLSMLAAGALLLSTSVMATNPPVDEPTDPGDSYARGP...,E9KJL1,NaN,3.1.1.74,NaN,NaN,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
573,210,unknown_enzyme,MRNTLKAAFKLGVISAALLAPFATQAATAGPGAWSSQQTWAADSVN...,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN
574,444,unknown_enzyme,NPYARGPNPTAASLEASAGPFTVRSFTVSRPSGYGAGTVYYPTNAG...,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN
575,445,unknown_enzyme,NPYARGPNPTAASLEASAGPFTVRSFTVSRPSGYGAGTVYYPTNAG...,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN
576,446,unknown_enzyme,NPYARGPNPTAASLEASAGPFTVRSFTVSRPSGYGAGTVYYPTNAG...,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN


In [11]:
lst = []
for pdb in df["pdb_id"].unique():
    lst.append(pdb)
lst

[nan,
 '7Z6B',
 '7VMD',
 '7VPA',
 '6SBN',
 '2FX5',
 '8AIS',
 '8AIT',
 '8SPK',
 '7DZT',
 '8AIR',
 '7CWG',
 '7EOA',
 '7QJQ',
 '5LUI',
 '5LUJ',
 '7QJT',
 '7QJO',
 '7QJP',
 '8A2C',
 '7NEI',
 '7CUV',
 '7YME',
 '7YM9',
 '7PZJ',
 '7QJM',
 '8B4U',
 '1TCA',
 '1AGY',
 '4OYL',
 '4EBO',
 '4CG1',
 '3A2P',
 '3A2Q',
 '2ZLY',
 '5XYO',
 '3AXG',
 '4PSU',
 '3VIS',
 '5D8M',
 '5KXVA',
 '4BTV',
 '2D81',
 '5AH1',
 '501l',
 '4B2N']

In [15]:
with open("input/unknown_pdb.txt", "w") as f:
    for l in lst:
        f.write(str(l)+'\n')

In [45]:
ids = df['protein_id'].unique().tolist()
ids

[nan,
 'Q8L3C9',
 'Q9UXF1',
 'P50381',
 'Q76HN6',
 'B2KJ46',
 'Q9P6C8',
 'Q6L0S1',
 'Q09669',
 'A1IIA3',
 'A1IIA4',
 'P39462',
 'P00326',
 'Q0PH30',
 'P08319',
 'Q6RS93',
 'Q9HIM3',
 'B2ZRE3',
 'B6UQD0',
 'A4IP64',
 'Q9QYY9',
 'P00330',
 'P00327',
 'P42328',
 'P00331',
 'B8QU18',
 'F9VMI9',
 'Q4J702',
 'Q4J9F2',
 'Q9ZT38',
 'Q1G151',
 'H9ZGN0',
 'Q8U259',
 'P86883',
 'P86884',
 'Q9Y9P9',
 'D4GSN2',
 'B0KBL1',
 'B0KBJ9',
 'G3FFC9',
 'V9SDP6',
 'V9SCJ1',
 'V9SFA1',
 'Q96XE0',
 'P0DJA2',
 'Q8GIX7',
 'Q8TZM9',
 'A3DCI2',
 'Q9CEN0',
 'Q5M4K4',
 'I3VSF1',
 'B0K3U7',
 'Q5JI09',
 'W6CX26',
 'B0K4A2',
 'C7R702',
 'Q8KLT9',
 'W8VSK8',
 'X514',
 'C7IV28',
 'Q70YJ9',
 'A0A060TBM3',
 'A3MVR8',
 'A0A1D8PP43',
 'E010289.1',
 'GenBank',
 'E7FHL9',
 'A1',
 'A0A075B5H4',
 'A0A1D3TV27',
 'G2I689',
 'A0A380WRS7',
 'P06525',
 'P74721',
 'A0A0U5FNC1',
 'A0A3T0KHC6',
 'Q1RS84',
 'P0A9Q7',
 'P28625',
 'Q7Z4W1',
 'Q70FD1',
 'Q91X52',
 'Q920P0',
 'Q920N9',
 'Q91XV4',
 'Q8NK50',
 'Q21929',
 'A0A0M4UR95',
 'A0A0E

In [5]:
API_URL = "https://rest.uniprot.org"
CCLLING_INTERVAL = 3 
retries = Retry(total=5, backoff_factor=0.25, status_forcelist=[500, 502, 503, 504])
session = requests.Session()
session.mount("https://", HTTPAdapter(max_retries=retries))

In [6]:
def check_response(response):
    try:
        response.raise_for_status()
    except requests.HTTPError:
        print(response.json())
        raise

In [7]:
def submit_id_mapping(from_db: str, to_db: str, ids: list):
    request = requests.post(
        f"{API_URL}/idmapping/run",
        data={"from": from_db, "to": to_db, "ids": ",".join(ids)},
    )
    check_response(request)
    return request.json()["jobId"]

In [8]:
ids = ids[1:50]
from_db = 'UniProtKB_AC-ID'
to_db = 'UniProtKB'


job_id = submit_id_mapping(from_db=from_db, to_db=to_db, ids=ids)
job_id

'7069edd265b6b954875407cc859f05c5c158d396'

In [9]:
def check_id_mapping_results_ready(job_id):
    while True:
        request = session.get(f"{API_URL}/idmapping/status/{job_id}")
        check_response(request)
        j = request.json()
        if "jobStatus" in j:
            if j["jobStatus"] == "RUNNING":
                print(f"Retrying in {POLLING_INTERVAL}s")
                time.sleep(POLLING_INTERVAL)
            else:
                raise Exception(j["jobStatus"])
        else:
            return bool(j["results"] or j["failedIds"])

In [10]:
if check_id_mapping_results_ready(job_id):
    print("ready")

ready


In [11]:
def get_id_mapping_results_search(url):
    parsed = urlparse(url)
    query = parse_qs(parsed.query)
    file_format = query["format"][0] if "format" in query else "json"
    if "size" in query:
        size = int(query["size"][0])
    else:
        size = 500
        query["size"] = size
    compressed = (
        query["compressed"][0].lower() == "true" if "compressed" in query else False
    )
    parsed = parsed._replace(query=urlencode(query, doseq=True))
    url = parsed.geturl()
    request = session.get(url)
    check_response(request)
    results = decode_results(request, file_format, compressed)
    total = int(request.headers["x-total-results"])
    print_progress_batches(0, size, total)
    for i, batch in enumerate(get_batch(request, file_format, compressed), 1):
        results = combine_batches(results, batch, file_format)
        print_progress_batches(i, size, total)
    if file_format == "xml":
        return merge_xml_results(results)
    return results

In [12]:
def get_id_mapping_results_link(job_id):
    url = f"{API_URL}/idmapping/details/{job_id}"
    request = session.get(url)
    check_response(request)
    return request.json()["redirectURL"]


In [13]:
def get_next_link(headers):
    re_next_link = re.compile(r'<(.+)>; rel="next"')
    if "Link" in headers:
        match = re_next_link.match(headers["Link"])
        if match:
            return match.group(1)

In [14]:
def get_batch(batch_response, file_format, compressed):
    batch_url = get_next_link(batch_response.headers)
    while batch_url:
        batch_response = session.get(batch_url)
        batch_response.raise_for_status()
        yield decode_results(batch_response, file_format, compressed)
        batch_url = get_next_link(batch_response.headers)

In [15]:
def print_progress_batches(batch_index, size, total):
    n_fetched = min((batch_index + 1) * size, total)
    print(f"Fetched: {n_fetched} / {total}")

In [16]:
def decode_results(response, file_format, compressed):
    if compressed:
        decompressed = zlib.decompress(response.content, 16 + zlib.MAX_WBITS)
        if file_format == "json":
            j = json.loads(decompressed.decode("utf-8"))
            return j
        elif file_format == "tsv":
            return [line for line in decompressed.decode("utf-8").split("\n") if line]
        elif file_format == "xlsx":
            return [decompressed]
        elif file_format == "xml":
            return [decompressed.decode("utf-8")]
        else:
            return decompressed.decode("utf-8")
    elif file_format == "json":
        return response.json()
    elif file_format == "tsv":
        return [line for line in response.text.split("\n") if line]
    elif file_format == "xlsx":
        return [response.content]
    elif file_format == "xml":
        return [response.text]
    return response.text

In [17]:
if check_id_mapping_results_ready(job_id):
    link = get_id_mapping_results_link(job_id)
    results = get_id_mapping_results_search(link)

Fetched: 49 / 49


In [ ]:
from scripts.uniprot import UniprotDownloader

ids = ids[1:50]
from_db = 'UniProtKB_AC-ID'
to_db = 'UniProtKB'

downloader = UniprotDownloader()

downloader.submit_id_mapping(from_db=from_db, to_db=to_db, ids=ids)



In [59]:
import json
s = json.dumps(results)
with open("tmp.json", "w") as f:
    f.write(s)

In [44]:
results['results'][0]['to']['proteinDescription']['recommendedName']['fullName']['value']

'alcohol dehydrogenase'

In [34]:
tmp = {}
references_list = []
for r in results['results'][0]['to']['references']:
    tmp["citationCrossReferences"] = r['citation']['citationCrossReferences']
    tmp.update({"title": r['citation']['title']})
    references_list.append(tmp)

print(str(references_list))

[{'citationCrossReferences': [{'database': 'PubMed', 'id': '17072683'}, {'database': 'DOI', 'id': '10.1007/s00792-006-0034-1'}], 'title': 'A cold-active and thermostable alcohol dehydrogenase of a psychrotorelant from Antarctic seawater, Flavobacterium frigidimaris KUC-1.'}]


In [35]:
go_ids = []
pfam_ids = []
for r in results['results'][0]['to']['uniProtKBCrossReferences']:
    if r['database'] == "GO":
        go_ids.append(r['id'])
    elif r['database'] == "pfam":
        pfam_ids.append(r['id'])
    else:
        continue
print(str(go_ids))

['GO:0005737', 'GO:0004022', 'GO:0008270']


In [36]:
results['results'][0]['to']['sequence']['value']

'MLPKTMKAAVIREFGSLLKIEEVEVKRPGRNEILVKVIASGVCHTDLHAVEGDWPVKPKMPLIPGHEAVGYVVAVGQEVKNVKEGDAVGVPWLYSACGGCDQCITGWETLCDTQQNGGYSVDGGFAEYVIADARYVGLLPSNVNFMEMAPILCAGVTVYKGLKETEVKPGEWVAISGIGGLGHVAVQYAKAMGMHVAAIDVADDKLDLAKKLGADLVVNAKNQNPGEFLKKEVGGMHGALITAVSPIAFKQGLETLRRKGTMALNGLPPGNFDLSIFDTVLNRITIRGSIVGTRKDMKEAIEFAVEGKVKATVTPAKLENINEVFDKMKKGQIEGRVVLEIAKA'

In [37]:
results['results'][0]['to']['sequence']['length']

344

In [39]:
results['results'][0]['to']['sequence']['molWeight']

36824

In [60]:
export_data = []
for result in results['results']:
    row = []
                
    row.append(result['from'])
                
    if result['to'] is not None:
        try:
            row.append(result['to']['proteinDescription']['recommendedName']['fullName']['value'])
        except KeyError:
            row.append("")

        tmp = {}
        references_list = []
                    
        try
            for r in result['to']['references']:
                tmp["citacionCrossReferences"] = r['citation']['citationCrossReferences']
                tmp.update({"title": r['citation']['title']})
                references_list.append(tmp)

        row.append(references_list)

        go_ids = []
        pfam_ids = []

        if result['to']['uniProtKBCrossReferences'] is not None:   
            for r in result['to']['uniProtKBCrossReferences']:
                if r['database'] == 'GO':
                    go_ids.append(r['id'])
                elif r['database'] == 'pfam':
                    pfam_ids.append(r['id'])
                else:
                    pass

        row.append(go_ids)
        row.append(pfam_ids)
                    
        row.append(result['to']['sequence']['value'])
        row.append(result['to']['sequence']['length'])
        row.append(result['to']['sequence']['molWeight'])
    else:
        row.append([result['from'], "", "", "", "", "", "", ""])

    export_data.append(row)


KeyError: 'recommendedName'

In [ ]:
for result in results['results']:
    print(result['from'])
    break

In [ ]:
for start in range(0, len(ids), 5):
    end = start + 5
    print(ids[start:end])